In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

from keras import layers, models, optimizers
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.models import Sequential

## Reading Data

In [ ]:
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train_data.head

In [ ]:
train_data.shape, test_data.shape

## Seperating Variables

In [ ]:
y = train_data['label']
x = train_data.drop(['label'], axis=1, inplace=True)

## Reshaping

In [ ]:
train_data = train_data.values.reshape(-1,28,28,1)
test_data = test_data.values.reshape(-1,28,28,1)
train_data.shape, test_data.shape

## Data Visualization

In [ ]:
plt.imshow(train_data[1])

In [ ]:
plt.figure(figsize=(20,20))
for i in range(10):
    plt.subplot(2, 10, i+1)
    plt.imshow(train_data[i])

In [ ]:
train = np.array(train_data)
y = np.array(y)
test = np.array(test_data)

In [ ]:
train.shape

## Digit Counting

In [ ]:
sns.countplot(y)
plt.title('digit count')

In [ ]:
y = to_categorical(y,num_classes=10) # it will do suppose y=4 ---> [0,0,0,0,1,0,0,0,0,0]

## Splitting Data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train, y, test_size = 0.2, random_state = 42)

## Data Augmentation

In [ ]:
data_gen = ImageDataGenerator(rescale = 1./255,
                              rotation_range=15,
                              width_shift_range=0.03,
                              height_shift_range=0.03,
                              shear_range=0.15,
                              fill_mode='nearest')
train_generator = data_gen.flow(x_train, y_train, batch_size=64)
val_generator = data_gen.flow(x_val, y_val, batch_size=64)

## Model Building

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = 3, strides=(1,1), padding='Same', activation='relu', input_shape = [28,28,1]))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = 3, strides=(2,2), padding='Same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters = 128, kernel_size = 3, strides=(1,1), padding='Same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = 3, strides=(2,2), padding='Same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.10))
model.add(Dense(10, activation="softmax"))

model.summary()

In [ ]:
optimizer = optimizers.Adam(learning_rate=0.0001)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

## Model Training

In [ ]:
batch_size=64
import keras
es_callback = keras.callbacks.EarlyStopping(monitor ='val_accuracy', patience = 10, mode = 'max', restore_best_weights = True)
reducelr_callback = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=8, min_lr=0.00000001)

history = model.fit_generator(train_generator, 
                             steps_per_epoch = len(x_train)//batch_size,
                             epochs=60,
                             validation_data = val_generator,
                             validation_steps = len(x_val)//batch_size,
                             callbacks = [es_callback, reducelr_callback])


## Plotting accuracy per epoch

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

ep = range(1,len(acc) + 1)

#Train and Validation accuracy
plt.plot(ep, acc, 'b', label='Training Accuracy')
plt.plot(ep, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

#Train and Validation loss
plt.plot(ep, loss, 'b', label='Training Loss')
plt.plot(ep, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

## Model Testing

In [ ]:
y_result = model.predict(test)
print('Prediction Shape is {}'.format(y_result.shape))

In [ ]:
results = np.argmax(y_result, axis=1)
results = pd.Series(results,name="Label")

## Submission

In [ ]:
My_submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

My_submission.to_csv("submission.csv",index=False)